In [416]:
import pandas as pd
import seaborn as sns
import numpy as np
import math
import matplotlib.pyplot as plt
from scipy import spatial
from sklearn import preprocessing
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD
from keras.utils import to_categorical
from keras.models import load_model

In [472]:
def get_direction(x1, y1, x2, y2):
    
    deg = math.degrees(np.arctan2(x2-x1,y2-y1))
    
    if(np.arctan2(x2-x1,y2-y1) < 0):
        deg+=360
    
    if(deg >= 22.5 and deg < 67.5):
        return 2
    elif(deg >= 67.5 and deg < 112.5):
        return 3
    elif(deg >= 112.5 and deg < 157.5):
        return 4
    elif(deg >= 157.5 and deg < 202.5):
        return 5
    elif(deg >= 202.5 and deg < 247.5):
        return 6
    elif(deg >= 247.5 and deg < 292.5):
        return 7
    elif(deg >= 292.5 and deg < 337.5):
        return 8
    elif(deg >= 337.5 or deg < 22.5):
        return 1

def ProcessRawData(df):
    df['stroke'] = list(pd.qcut(df['timestamp'], len(df)//30, labels=np.arange(0, len(df)//30)))
    
    distX = df.loc[:, 'x'].diff()
    distX[0] = 0
    distY = df.loc[:, 'y'].diff()
    distY[0] = 0
    distTime = df.loc[:, 'timestamp'].diff()
    distTime[0] = 0
    df['distance'] = np.sqrt(np.square(distX) + np.square(distY))
    df['velocity'] = df['distance']/distTime
    df.loc[0, 'velocity'] = 0
    
    velocityList = ['Velocity_Total', 'Velocity_Min', 'Velocity_Max', 'Velocity_STD', 'Velocity_Mean', 'Velocity_Median']
    columns = velocityList
    TrainingData = pd.DataFrame(columns=columns, index=list(np.arange(1, 9)))
    
    df['direction'] = -1
    for x in df['stroke'].unique():
        direction = get_direction(int(df[df['stroke'] == x].head(1)['x']),
            int(df[df['stroke'] == x].head(1)['y']),
            int(df[df['stroke'] == x].tail(1)['x']),
            int(df[df['stroke'] == x].tail(1)['y'])
           )
        df.loc[df['stroke'] == x, 'direction'] = direction
    
    timeStampKeyList = []
    for x in range(0, len(df)):
        if(x == 0):
            continue
        if(df.loc[x, 'keyDown'] == 1 and df.loc[x-1, 'keyDown'] == 1):
            timeStampKeyList.append(df.loc[x, 'timestamp'] - df.loc[x-1, 'timestamp'])
    timeStampKeyList = np.array(timeStampKeyList)
    
    timeStampClickList = []
    for x in range(0, len(df)):
        
        if(df.loc[x, 'mouseDown'] == 256):
            timeStampClickList.append(df.loc[x, 'timestamp'] - df.loc[x-1, 'timestamp'])
    timeStampClickList = np.array(timeStampClickList)
    
    
    for x in df['direction'].unique():
        TrainingData.loc[x, 'Velocity_Total'] = df[df['direction'] == x]['velocity'].sum()
        TrainingData.loc[x, 'Velocity_Min'] = df[df['direction'] == x]['velocity'].min()
        TrainingData.loc[x, 'Velocity_Max'] = df[df['direction'] == x]['velocity'].max()
        TrainingData.loc[x, 'Velocity_STD'] = df[df['direction'] == x]['velocity'].std()
        TrainingData.loc[x, 'Velocity_Mean'] = df[df['direction'] == x]['velocity'].mean()
        TrainingData.loc[x, 'Velocity_Median'] = df[df['direction'] == x]['velocity'].median()
    
    X = np.array(TrainingData).flatten()
    
    MouseData = np.array([timeStampClickList.sum(),
                     timeStampClickList.min(),
                     timeStampClickList.max(),
                     timeStampClickList.std(),
                     timeStampClickList.mean(),
                     np.median(timeStampClickList)])

    KeyboardData = np.array([timeStampKeyList.sum(),
                     timeStampKeyList.min(),
                     timeStampKeyList.max(),
                     timeStampKeyList.std(),
                     timeStampKeyList.mean(),
                     np.median(timeStampKeyList)])
    
    result = np.concatenate((X, MouseData, KeyboardData))
    
    #Normalize Result
    return(result)
    
    

In [477]:
df = pd.read_csv('../datasets/JAR60')
df['group'] = pd.cut(x=df['timestamp'], bins=6, labels=list(np.arange(0, 6)))

JosephIntervals = []

for x in df['group'].unique():
    clone = df.loc[df['group'] == x, :].copy()
    clone = clone.drop('group', axis=1).reset_index()
    JosephIntervals.append(ProcessRawData(clone))
    
df = pd.read_csv('../datasets/CSJ30')
df['group'] = pd.cut(x=df['timestamp'], bins=6, labels=list(np.arange(0, 6)))

ChristiansIntervals = []

for x in df['group'].unique():
    clone = df.loc[df['group'] == x, :].copy()
    clone = clone.drop('group', axis=1).reset_index()
    ChristiansIntervals.append(ProcessRawData(clone))

In [478]:
x_train = np.array((JosephIntervals[0], JosephIntervals[1], JosephIntervals[2], JosephIntervals[3], JosephIntervals[4],JosephIntervals[5],
                    ChristiansIntervals[0], ChristiansIntervals[1], ChristiansIntervals[2], ChristiansIntervals[3], ChristiansIntervals[4], ChristiansIntervals[5]))
y_train = np.array([[1, 0], [1, 0], [1, 0], [1, 0], [1,0], [1,0], [0, 1], [0,1] , [0,1], [0,1], [0, 1], [0, 1]])

x_train

array([[710.1196305489541, 0.0, 4.428571428571429, 0.5727975339506444,
        0.37712141824161133, 0.13975424859373686, 1986.4516071188834,
        0.0, 6.437735971942655, 1.2249356908297218, 0.9912433169255905,
        0.5, 6645.825919261314, 0.0, 62.80127387243033,
        1.6480324838916023, 1.1852730371430915, 0.625,
        2151.6553553641347, 0.0, 10.73312629199899, 1.4039563314331143,
        1.0096927993262013, 0.5150787536377127, 1064.3858166537034, 0.0,
        4.989785484735137, 0.6973654726442403, 0.60100836626409, 0.375,
        2442.699014664565, 0.0, 22.14158982548453, 1.44349918155564,
        1.0713592169581425, 0.5590169943749475, 5466.967420698568, 0.0,
        32.2490309931942, 1.2676920837870467, 1.0713242055062842, 0.625,
        3292.3343773641877, 0.0, 12.40071682515885, 1.3544899351168793,
        1.154799851758747, 0.6388765649999399, 26181.0, 1.0, 1440.0,
        107.51004940348098, 48.84514925373134, 8.0, 140497.0, 16.0,
        1753.0, 132.10055350158805, 

In [479]:
model = Sequential()
model.add(Dense(64, activation='relu', input_dim=60))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(2, activation='softmax'))

In [480]:
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])

#Train our model for 100 iteration of the training data
model.fit(x_train, y_train, epochs=100, shuffle=True, validation_split=0.2)

Train on 9 samples, validate on 3 samples
Epoch 1/100
9/9 [==============================] - 4s 447ms/step - loss: 7.1246 - acc: 0.5556 - val_loss: 1.0960e-07 - val_acc: 1.0000
Epoch 2/100
9/9 [==============================] - 0s 535us/step - loss: 7.1246 - acc: 0.5556 - val_loss: 1.0960e-07 - val_acc: 1.0000
Epoch 3/100
9/9 [==============================] - 0s 588us/step - loss: 7.1246 - acc: 0.5556 - val_loss: 1.0960e-07 - val_acc: 1.0000
Epoch 4/100
9/9 [==============================] - 0s 986us/step - loss: 7.1246 - acc: 0.5556 - val_loss: 1.0960e-07 - val_acc: 1.0000
Epoch 5/100
9/9 [==============================] - 0s 698us/step - loss: 7.1246 - acc: 0.5556 - val_loss: 1.0960e-07 - val_acc: 1.0000
Epoch 6/100
9/9 [==============================] - 0s 655us/step - loss: 7.1246 - acc: 0.5556 - val_loss: 1.0960e-07 - val_acc: 1.0000
Epoch 7/100
9/9 [==============================] - 0s 929us/step - loss: 7.1246 - acc: 0.5556 - val_loss: 1.0960e-07 - val_acc: 1.0000
Epoch 8/100
9

9/9 [==============================] - 0s 650us/step - loss: 7.1246 - acc: 0.5556 - val_loss: 1.0960e-07 - val_acc: 1.0000
Epoch 62/100
9/9 [==============================] - 0s 805us/step - loss: 7.1246 - acc: 0.5556 - val_loss: 1.0960e-07 - val_acc: 1.0000
Epoch 63/100
9/9 [==============================] - 0s 628us/step - loss: 7.1246 - acc: 0.5556 - val_loss: 1.0960e-07 - val_acc: 1.0000
Epoch 64/100
9/9 [==============================] - 0s 1ms/step - loss: 7.1246 - acc: 0.5556 - val_loss: 1.0960e-07 - val_acc: 1.0000
Epoch 65/100
9/9 [==============================] - 0s 477us/step - loss: 7.1246 - acc: 0.5556 - val_loss: 1.0960e-07 - val_acc: 1.0000
Epoch 66/100
9/9 [==============================] - 0s 425us/step - loss: 7.1246 - acc: 0.5556 - val_loss: 1.0960e-07 - val_acc: 1.0000
Epoch 67/100
9/9 [==============================] - 0s 552us/step - loss: 7.1246 - acc: 0.5556 - val_loss: 1.0960e-07 - val_acc: 1.0000
Epoch 68/100
9/9 [==============================] - 0s 765us/st

In [435]:
#model.save('Koios.h5')

In [463]:
josephTest = pd.read_csv('../datasets/JosephReal5B')
model.predict(JosephIntervals[4].reshape(1, -1))

array([[0., 1.]], dtype=float32)